# Spark SQL

### import and create a SparkSession

In [4]:
from pyspark.sql import SparkSession
spark1 = SparkSession.builder.appName('SQL').getOrCreate()

### create a dataframe from csv file

In [6]:
df = spark1.read.csv('appl_data.csv',header=True,inferSchema=True)

In [17]:
df

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

In [7]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



### Create a temporary view on the Dataframe to perform Spark SQL queries on it

In [8]:
df.createOrReplaceTempView('stock')

### The result from the SQL query returns a dataframe

In [9]:
result = spark1.sql('SELECT * FROM stock LIMIT 5')

In [10]:
result

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

In [11]:
result.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [12]:
result.show()

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+-------------------+----------+----------+------------------+------------------+---------+------------------+



### Spark SQL queries examples

In [13]:
count_greater_500 = spark1.sql('SELECT count(*) FROM stock WHERE Close > 500')

In [14]:
count_greater_500.show()

+--------+
|count(1)|
+--------+
|     403|
+--------+



In [15]:
avg_1 = spark1.sql('SELECT avg(OPEN) from stock WHERE Volume > 120000000 or Volume < 110000000')

In [16]:
avg_1.show()

+------------------+
|         avg(OPEN)|
+------------------+
|309.12406365290224|
+------------------+



### create a dataframe from an external file using Spark SQL

In [20]:
df_sales = spark1.sql("SELECT * FROM csv.`sales.csv`")

In [21]:
df_sales.show()

+-------+-------+-----+
|    _c0|    _c1|  _c2|
+-------+-------+-----+
|Company| Person|Sales|
|   GOOG|    Sam|  200|
|   GOOG|Charlie|  120|
|   GOOG|  Frank|  340|
|   MSFT|   Tina|  600|
|   MSFT|    Amy|  124|
|   MSFT|Vanessa|  243|
|     FB|   Carl|  870|
|     FB|  Sarah|  350|
|   APPL|   John|  250|
|   APPL|  Linda|  130|
|   APPL|   Mike|  750|
|   APPL|  Chris|  350|
+-------+-------+-----+



### Reading from a local sqllite DB using JDBC connection

#### install SQLlite jar file in SPARK_HOME

In [23]:
# cd $SPARK_HOME
# curl https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.28.0/sqlite-jdbc-3.28.0.jar

#### Define driver, path to local .db file, and append that path to jdbc:sqlite to construct the url

In [1]:
driver = "org.sqlite.JDBC"
path = "chinook.db"
url = "jdbc:sqlite:" + path

#### specify the table to be read

In [2]:
tablename = "albums"

In [5]:
df_albums = spark1.read.format("jdbc").option("url",url).option("dbtable",tablename).option("driver",driver).load()

In [6]:
df_albums.show()

+-------+--------------------+--------+
|AlbumId|               Title|ArtistId|
+-------+--------------------+--------+
|      1|For Those About T...|       1|
|      2|   Balls to the Wall|       2|
|      3|   Restless and Wild|       2|
|      4|   Let There Be Rock|       1|
|      5|            Big Ones|       3|
|      6|  Jagged Little Pill|       4|
|      7|            Facelift|       5|
|      8|      Warner 25 Anos|       6|
|      9|Plays Metallica B...|       7|
|     10|          Audioslave|       8|
|     11|        Out Of Exile|       8|
|     12| BackBeat Soundtrack|       9|
|     13|The Best Of Billy...|      10|
|     14|Alcohol Fueled Br...|      11|
|     15|Alcohol Fueled Br...|      11|
|     16|       Black Sabbath|      12|
|     17|Black Sabbath Vol...|      12|
|     18|          Body Count|      13|
|     19|    Chemical Wedding|      14|
|     20|The Best Of Buddy...|      15|
+-------+--------------------+--------+
only showing top 20 rows



In [7]:
df_albums.printSchema()

root
 |-- AlbumId: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- ArtistId: integer (nullable = true)



In [8]:
df_albums.createOrReplaceTempView('albums')

#### read another table 

In [9]:
df_artists = spark1.read.format("jdbc").option("url",url).option("dbtable",'artists').option("driver",driver).load()

In [10]:
df_artists.show()

+--------+--------------------+
|ArtistId|                Name|
+--------+--------------------+
|       1|               AC/DC|
|       2|              Accept|
|       3|           Aerosmith|
|       4|   Alanis Morissette|
|       5|     Alice In Chains|
|       6|Antônio Carlos Jobim|
|       7|        Apocalyptica|
|       8|          Audioslave|
|       9|            BackBeat|
|      10|        Billy Cobham|
|      11| Black Label Society|
|      12|       Black Sabbath|
|      13|          Body Count|
|      14|     Bruce Dickinson|
|      15|           Buddy Guy|
|      16|      Caetano Veloso|
|      17|       Chico Buarque|
|      18|Chico Science & N...|
|      19|        Cidade Negra|
|      20|        Cláudio Zoli|
+--------+--------------------+
only showing top 20 rows



In [11]:
df_artists.createOrReplaceTempView('artists')

### Join artists and albums into a single dataframe

In [12]:
df_combined = spark1.sql("SELECT * FROM albums LEFT JOIN artists ON albums.ArtistId=artists.ArtistId order by artists.ArtistId")

In [13]:
df_combined.show()

+-------+--------------------+--------+--------+--------------------+
|AlbumId|               Title|ArtistId|ArtistId|                Name|
+-------+--------------------+--------+--------+--------------------+
|      1|For Those About T...|       1|       1|               AC/DC|
|      4|   Let There Be Rock|       1|       1|               AC/DC|
|      2|   Balls to the Wall|       2|       2|              Accept|
|      3|   Restless and Wild|       2|       2|              Accept|
|      5|            Big Ones|       3|       3|           Aerosmith|
|      6|  Jagged Little Pill|       4|       4|   Alanis Morissette|
|      7|            Facelift|       5|       5|     Alice In Chains|
|      8|      Warner 25 Anos|       6|       6|Antônio Carlos Jobim|
|     34|Chill: Brazil (Di...|       6|       6|Antônio Carlos Jobim|
|      9|Plays Metallica B...|       7|       7|        Apocalyptica|
|    271|         Revelations|       8|       8|          Audioslave|
|     11|        Out

### Global and temporary views

#### A temporary view does not persist across multiple sessions (SparkSession)

In [14]:
df_artists.createOrReplaceTempView("temp_artists")

df_temp = spark1.sql("SELECT * FROM temp_artists LIMIT 10")
df_temp.show()

+--------+--------------------+
|ArtistId|                Name|
+--------+--------------------+
|       1|               AC/DC|
|       2|              Accept|
|       3|           Aerosmith|
|       4|   Alanis Morissette|
|       5|     Alice In Chains|
|       6|Antônio Carlos Jobim|
|       7|        Apocalyptica|
|       8|          Audioslave|
|       9|            BackBeat|
|      10|        Billy Cobham|
+--------+--------------------+



In [15]:
spark2 = SparkSession.builder.appName('SQL2').getOrCreate()

In [16]:
try:
    df_temp = spark2.sql("SELECT * FROM temp_artists LIMIT 10")
except:
    print("Error happened in this execution")

#### Global views can persist across multiple sessions